    16_230110_117_happyhouse
     4-3 section
     
     data : C:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/data/happy_house.csv

In [7]:
#1. package load
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap
print('슝~')

슝~


In [8]:
#2. data load  ...SGIS data 가져오기 # SGIS API 사이트의 본인의 consumer_key, consumer_secret를 입력합니다.
consumer_key = '2f74504a63224ee09f2b'       # 서비스ID
consumer_secret =  '63a0ffef893e43a48ae2' # 보안Key


In [9]:
#2. data load  ... SGIS data 가져오기 # SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
   
    return access_token
print('슝~')

슝~


In [10]:
#9 Folium를 활용한 공간 데이터 시각화
### 전국도시철도역사정보표준데이터 가져오기
                                    
apt_df = pd.read_csv("C:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/data/happy_house.csv")
apt_df.head()

,지역,단지,세대수,주소,위도,경도
0,고양시 덕양구,삼송11단지,832,경기도 고양시 덕양구 원흥1로 25,37.650992,126.865491
1,고양시 덕양구,고양지축 A-1블록 신혼희망타운 행복주택,250,경기도 고양시 덕양구 오부자로 99,37.653331,126.921037
2,고양시 덕양구,고양지축 A-4BL 행복주택,473,경기도 고양시 덕양구 오부자로 43,37.654038,126.914946
3,고양시 덕양구,고양삼송 LH원흥마을13단지,947,경기도 고양시 덕양구 원흥1로 35,37.652631,126.868888
4,고양시 덕양구,서정마을LH12단지,276,경기도 고양시 덕양구 서정마을2로 17,37.621215,126.847648


In [11]:
apt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지역      95 non-null     object 
 1   단지      95 non-null     object 
 2   세대수     95 non-null     int64  
 3   주소      95 non-null     object 
 4   위도      95 non-null     float64
 5   경도      95 non-null     float64
dtypes: float64(2), int64(1), object(3)
memory usage: 4.6+ KB


In [16]:
#5 seoul university   # 주소를 입력해 주소의 위도, 경도를 가지고 오는 함수
def get_adress_x_y(address) -> None:
    # SGIS access_token 가져오기
    access_token = get_access_token(consumer_key , consumer_secret)

    url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params={
        "accessToken": access_token,
        "address": address
    }

    temp_res = requests.get(
        url=url,
        params=params
    ).json()["result"]["resultdata"]

    return temp_res[0]["x"], temp_res[0]["y"]

print('슝~')

슝~


In [18]:
#5 seoul university .. 주소 기반 위도,경도 추가  실패건수 3건
#경기도 남양주시 경춘로 899은(는) 검색에 실패했습니다
#경기도 파주시 노을빛로 41은(는) 검색에 실패했습니다

cnt = 0
for _, row in apt_df.iterrows():
    try:
        x, y = get_adress_x_y(row["주소"])
     
        apt_df.loc[_, "x"] = x
        apt_df.loc[_, "y"] = y
    except:
        print(row["주소"] + '은(는) 검색에 실패했습니다')
        cnt = cnt + 1

print("실패건수 : ", cnt)
apt_df.head()


경기도 남양주시 경춘로 899은(는) 검색에 실패했습니다
경기도 파주시 노을빛로 41은(는) 검색에 실패했습니다
non은(는) 검색에 실패했습니다
실패건수 :  3


,지역,단지,세대수,주소,위도,경도,x,y
0,고양시 덕양구,삼송11단지,832,경기도 고양시 덕양구 원흥1로 25,37.650992,126.865491,944033.138639635,1961509.74797967
1,고양시 덕양구,고양지축 A-1블록 신혼희망타운 행복주택,250,경기도 고양시 덕양구 오부자로 99,37.653331,126.921037,948927.188681021,1961563.84052786
2,고양시 덕양구,고양지축 A-4BL 행복주택,473,경기도 고양시 덕양구 오부자로 43,37.654038,126.914946,948402.245221731,1961752.3222608
3,고양시 덕양구,고양삼송 LH원흥마을13단지,947,경기도 고양시 덕양구 원흥1로 35,37.652631,126.868888,944239.664818954,1961599.57409593
4,고양시 덕양구,서정마을LH12단지,276,경기도 고양시 덕양구 서정마을2로 17,37.621215,126.847648,942442.99302673,1958197.02299515


In [19]:
#5 seoul university  ..# SGIS 좌표변환 API를 활용한 좌표변환 함수
def get_transcoord(src, dst, posX, posY) -> None:
    access_token = get_access_token(consumer_key, consumer_secret)
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"] # 변환된 좌표값 딕셔너리의 value 리턴

print('슝~')


슝~


In [20]:
#5 seoul university  .. 좌표계가 UTM-K이므로 WSG84좌표계로 변환
for _, row in apt_df.iterrows():
    try:
        x, y = get_transcoord("EPSG:5179", "EPSG:4326", row["x"], row["y"])
    
        apt_df.loc[_, "lng"] = x
        apt_df.loc[_, "lat"] = y

    except:
        pass
apt_df.head()

,지역,단지,세대수,주소,위도,경도,x,y,lng,lat
0,고양시 덕양구,삼송11단지,832,경기도 고양시 덕양구 원흥1로 25,37.650992,126.865491,944033.138639635,1961509.74797967,126.865540,37.651375
1,고양시 덕양구,고양지축 A-1블록 신혼희망타운 행복주택,250,경기도 고양시 덕양구 오부자로 99,37.653331,126.921037,948927.188681021,1961563.84052786,126.921014,37.652148
2,고양시 덕양구,고양지축 A-4BL 행복주택,473,경기도 고양시 덕양구 오부자로 43,37.654038,126.914946,948402.245221731,1961752.3222608,126.915050,37.653817
3,고양시 덕양구,고양삼송 LH원흥마을13단지,947,경기도 고양시 덕양구 원흥1로 35,37.652631,126.868888,944239.664818954,1961599.57409593,126.867874,37.652197
4,고양시 덕양구,서정마을LH12단지,276,경기도 고양시 덕양구 서정마을2로 17,37.621215,126.847648,942442.99302673,1958197.02299515,126.847776,37.621419


In [27]:
#5 seoul university  .. 시각화 .. 초기 중심값 좌표가 들어가야 합니다.

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 12로 시작합니다.
map = folium.Map(
    location=center,
    zoom_start=12

)

print('슝~')

슝~


In [32]:
#2. 프로젝트(1)  # MarkerCluster 객체 
marker_cluster = MarkerCluster().add_to(map)

#maker 객체 추가
for idx, row in apt_df.iterrows():
    try:
        folium.Marker(
            location=[apt_df.loc[idx, "lat"], apt_df.loc[idx, "lng"]],
            popup=apt_df.loc[idx, "단지"],
            icon=folium.Icon(color="cadetblue", icon="house", prefix="fa")            
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

슝~


In [33]:
#5 seoul university 
map

In [24]:
map.save("happy_house.html")

In [25]:
apt_df.to_csv("happy_house.csv", index = False)

In [26]:
# #2. 프로젝트(1)  # MarkerCluster 객체 
# marker_cluster = MarkerCluster().add_to(map)

# #maker 객체 추가
# for idx, row in apt_df.iterrows():
#     try:
#         folium.Marker(
#             location=[apt_df.loc[idx, "lat"], apt_df.loc[idx, "lng"]],
#             popup=apt_df.loc[idx, "공동주택명정보"],
#             icon=folium.Icon(color="cadetblue", icon="car", prefix="fa")            
#         ).add_to(marker_cluster)
#     except:
#         pass

# print('슝~')